This note is used for debugging Tony's perturb pipeline 

In [2]:
import pandas as pd
import sys
print(sys.executable)

/oak/stanford/groups/engreitz/Users/shshanhe/tools/miniforge3/envs/torch-cNMF/bin/python


# bug in read excel

In [10]:
df_k562=pd.read_excel("references_yann_k562/sample_info.k562_yann.xlsx")
df_eRZ_20A=pd.read_excel("/oak/stanford/groups/engreitz/Users/ronghao/Sequencing/20251218_eRZ59_Split-seq_4M_Novaseq/perturb_pipeline/analysis/references_eRZ59/sample_info_eRZ58_20A.xlsx")
df_eRZ_40B=pd.read_excel("/oak/stanford/groups/engreitz/Users/ronghao/Sequencing/20251218_eRZ59_Split-seq_4M_Novaseq/perturb_pipeline/analysis/references_eRZ59/sample_info_eRZ58_40B.xlsx")
df_k562
df_eRZ_20A
df_eRZ_40B

,pool,sample,sample_id,sample_type,expected_cells,min_umi_threshold,fastq_dir,sample_to_well_mapping,i7_barcode,i5_barcode,notes,read1_length,read2_length,paired_guide_pool,paired_guide_sample,paired_guide_sample_id
0,eRZ58,eRZ58_GEX40B,eRZ58:eRZ58_GEX40B,gex,5000,50,/oak/stanford/groups/engreitz/Users/ronghao/Se...,plate1,CTTGTA,CTGGCA,NaN,64,58,eRZ58,eRZ58_gRNA40B,eRZ58:eRZ58_gRNA40B
1,eRZ58,eRZ58_gRNA40B,eRZ58:eRZ58_gRNA40B,guide,5000,50,/oak/stanford/groups/engreitz/Users/ronghao/Se...,plate1,TCGGCA,GGTAGC,NaN,64,58,NaN,NaN,NaN


# bug in ribosomal anotation, need to be modified

In [8]:
#import gffutils
from collections import Counter
import random
import sys
!{sys.executable} -m pip install gffutils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [gffutils]4/5 [gffutils]te]


In [12]:
import gffutils

gene_database_path = "/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/Mus_musculus.GRCm39.108.annotation.db"
db = gffutils.FeatureDB(gene_database_path)
g = next(db.features_of_type("gene"))
print("ID:", g.id)
# ID: ENSMUSG00000102628
print("seqid:", g.seqid, "start:", g.start, "end:", g.end, "strand:", g.strand)
# seqid: 1 start: 150956201 end: 150958296 strand: +
print("attribute keys:", sorted(g.attributes.keys()))
gene_id = "ENSMUSG00000102628"
g
print("gene_biotype:", g.attributes.get("gene_biotype", [""])[0])
print("gene_id:", g.attributes.get("gene_id", [""])[0])
print("gene_type:", g.attributes.get("gene_type", [""])[0])
print("hgnc_id:", g.attributes.get("hgnc_id",  [""])[0])
print("gene_name:", g.attributes.get("gene_name",[""])[0])


ID: ENSMUSG00000102628
seqid: 1 start: 150956201 end: 150958296 strand: +
attribute keys: ['gene_biotype', 'gene_id', 'gene_name', 'gene_source', 'gene_version']
gene_biotype: TEC
gene_id: ENSMUSG00000102628
gene_type: 
hgnc_id: 
gene_name: Gm37671


In [20]:
# human
gene_database_path = "/oak/stanford/groups/engreitz/Users/tonyzeng/GW_PERTURB/references/gencode.v46.annotation.db"
db = gffutils.FeatureDB(gene_database_path)
g = next(db.features_of_type("gene"))
print("ID:", g.id)
print("seqid:", g.seqid, "start:", g.start, "end:", g.end, "strand:", g.strand)
print("attribute keys:", sorted(g.attributes.keys()))

gene_id = "ENSG00000142541"
g
print("gene_biotype:", g.attributes.get("gene_biotype", [""])[0])
print("gene_id:", g.attributes.get("gene_id", [""])[0])
print("gene_type:", g.attributes.get("gene_type", [""])[0])
print("hgnc_id:", g.attributes.get("hgnc_id",  [""])[0])
print("gene_name:", g.attributes.get("gene_name",[""])[0])

ID: ENSG00000290825.1
seqid: chr1 start: 11869 end: 14409 strand: +
attribute keys: ['gene_id', 'gene_name', 'gene_type', 'level', 'tag']
gene_biotype: 
gene_id: ENSG00000290825.1
gene_type: lncRNA
hgnc_id: 
gene_name: DDX11L2


In [8]:
def add_comprehensive_gene_annotations(adata, ribosomal_genes_path, gene_database_path, cell_cycle_genes_path):
    """Add comprehensive gene annotations to var dataframe.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    ribosomal_genes_path : str
        Path to ribosomal genes file
    gene_database_path : str
        Path to gene database file
    cell_cycle_genes_path : str
        Path to cell cycle genes file
    """
    log_print("🧬 Adding comprehensive gene annotations...")

    # Create reference files dict for internal use
    REFERENCE_FILES = {
        "ribosomal_genes": ribosomal_genes_path,
        "gene_database": gene_database_path,
        "cell_cycle_genes": cell_cycle_genes_path
    }

    # Load ribosomal genes from tab-separated file - use "HGNC ID" column
    # ribosomal_hgnc_ids = set()
    with open(REFERENCE_FILES["ribosomal_genes"]) as f:
        ribosomal_genes_list = [x.strip() for x in f]
    ribosomal_genes = set(ribosomal_genes_list)  # For fast lookup

        # header = f.readline().strip().split('\t')
        # hgnc_col = header.index("HGNC ID")
        # for line in f:
        #     cols = line.strip().split('\t')
        #     if len(cols) > hgnc_col:
        #         ribosomal_hgnc_ids.add(cols[hgnc_col])

    log_print(f"  📋 Loaded {len(ribosomal_genes)} ribosomal gene")
    log_print(f"  📋 Sample ribosomal gene: {list(ribosomal_genes)[:5]}")

    # Load gencode database
    db = gffutils.FeatureDB(REFERENCE_FILES["gene_database"])

    # Initialize annotation columns
    adata.var['gene_type'] = ''
    # adata.var['hgnc_id'] = ''
    adata.var['gene'] = ''  # Gene symbols
    adata.var['chromosome'] = ''
    adata.var['ribosomal'] = False
    adata.var['cell_cycle'] = False
    adata.var['cell_cycle_phase'] = ''
    adata.var['pct_cells_expressed'] = 0.0

    # Calculate % cells expressed (using expression threshold > 0)
    log_print("  📊 Calculating % cells expressed...")
    cells_expressed = (adata.X > 0).sum(axis=0)
    if hasattr(cells_expressed, 'A1'):
        cells_expressed = cells_expressed.A1
    adata.var['pct_cells_expressed'] = (cells_expressed / adata.n_obs) * 100

    # Load cell cycle genes for annotation
    with open(REFERENCE_FILES["cell_cycle_genes"]) as f:
        cell_cycle_genes_list = [x.strip() for x in f]
    cell_cycle_genes = set(cell_cycle_genes_list)  # For fast lookup
    s_phase_genes = set(cell_cycle_genes_list[:43])  # First 43 are S phase

    # Annotate from gencode database first
    log_print("  🗃️  Loading gene annotations from gencode database...")
    log_print(f"  🔍 Sample gene IDs from adata.var.index: {list(adata.var.index[:5])}")

    annotated_count = 0
    missing_count = 0
    error_count = 0

    for i, gene_id in enumerate(adata.var.index):
        if i < 5:  # Debug first 5 genes
            log_print(f"  🔍 Trying to annotate gene {i+1}: {gene_id}")

        try:
            # Query database for this gene by ID
            gene = db[gene_id]
        except gffutils.exceptions.FeatureNotFoundError:
            gene = db[gene_id.split(".", 1)[0]]
            
            if i < 5:
                log_print(f"    ✅ Found gene {gene_id} in database")

            # Extract annotations - handle missing attributes gracefully
            # NOTE: We use .get() here because not all genes have all attributes (e.g., some lack hgnc_id)
            # This is expected and OK - we should NOT raise errors for missing optional attributes
            gene_type = gene.attributes.get('gene_biotype', [''])[0]
            #hgnc_id = gene.attributes.get('hgnc_id', [''])[0]
            gene_name = gene.attributes.get('gene_name', [''])[0]

            if i < 5:
                log_print(f"    📋 Gene info: type={gene_type}, name={gene_name}, chr={gene.seqid}")

            # Update annotations
            adata.var.loc[gene_id, 'gene_type'] = gene_type
            # adata.var.loc[gene_id, 'hgnc_id'] = hgnc_id
            adata.var.loc[gene_id, 'gene'] = gene_name  # Add gene symbols
            adata.var.loc[gene_id, 'chromosome'] = gene.seqid

            # Check ribosomal genes (by HGNC ID)
            if gene_name in ribosomal_genes:
                adata.var.loc[gene_id, 'ribosomal'] = True

            # Check cell cycle genes (by gene name)
            if gene_name in cell_cycle_genes:
                adata.var.loc[gene_id, 'cell_cycle'] = True
                # Determine phase using pre-computed S phase set
                if gene_name in s_phase_genes:
                    adata.var.loc[gene_id, 'cell_cycle_phase'] = 'S'
                else:
                    adata.var.loc[gene_id, 'cell_cycle_phase'] = 'G2M'

            annotated_count += 1

        except Exception as e:
            raise RuntimeError(f"Failed to annotate gene {gene_id}: {e}") from e

    log_print(f"  📊 Annotation results: {annotated_count} success, {missing_count} missing, {error_count} errors")

    # CRITICAL: Fail if gene annotation completely failed
    if annotated_count == 0:
        raise RuntimeError(f"CRITICAL: Gene annotation completely failed. 0/{adata.n_vars} genes annotated. Database lookup is broken.")

    # CRITICAL: Fail if most genes failed annotation
    success_rate = annotated_count / adata.n_vars
    if success_rate < 0.5:  # Less than 50% success
        raise RuntimeError(f"CRITICAL: Gene annotation mostly failed. Only {success_rate:.1%} ({annotated_count}/{adata.n_vars}) genes annotated successfully.")

    # Calculate mitochondrial genes using annotated gene symbols
    log_print("  🧬 Calculating mitochondrial gene scores...")
    mt_genes = adata.var["gene"].str.startswith(("MT-", "mt-"), na=False)
    log_print(f"  📊 Found {mt_genes.sum()} mitochondrial genes")

    if mt_genes.sum() > 0:
        mt_gene_names = adata.var_names[mt_genes].tolist()

        log_print(f"  ✅ Added MT percentage and score using {len(mt_gene_names)} genes")
    else:
        raise RuntimeError("CRITICAL: No mitochondrial genes found for scoring. MT genes are required for quality control.")

    # Calculate ribosomal scores using annotated genes
    log_print("  🧬 Calculating ribosomal gene scores...")
    ribo_genes = adata.var['ribosomal'] == True
    log_print(f"  📊 Found {ribo_genes.sum()} ribosomal genes")

    if ribo_genes.sum() > 0:
        ribo_gene_names = adata.var_names[ribo_genes].tolist()

        # Calculate ribosomal percentage
        adata.obs['pct_counts_ribo'] = (adata[:, ribo_genes].X.sum(axis=1) / adata.X.sum(axis=1)) * 100
        if hasattr(adata.obs['pct_counts_ribo'], 'A1'):
            adata.obs['pct_counts_ribo'] = adata.obs['pct_counts_ribo'].A1

        log_print(f"  ✅ Added ribosomal percentage and score using {len(ribo_gene_names)} genes")
    else:
        raise RuntimeError("CRITICAL: No ribosomal genes found for scoring. Ribosomal gene metrics are required for quality control.")

    # Summary statistics
    log_print(f"  ✅ Annotated {annotated_count}/{adata.n_vars} genes")
    log_print(f"  📊 Gene types: {adata.var['gene_type'].value_counts().head().to_dict()}")
    log_print(f"  📊 Ribosomal: {adata.var['ribosomal'].sum()}")
    log_print(f"  📊 Cell cycle: {adata.var['cell_cycle'].sum()}")
    log_print(f"  📊 Chromosomes: {adata.var['chromosome'].nunique()} unique")
    log_print(f"  📊 Mean % cells expressed: {adata.var['pct_cells_expressed'].mean():.1f}%")


In [13]:
def add_comprehensive_gene_annotations(adata, ribosomal_genes_path, gene_database_path, cell_cycle_genes_path):
    """Add comprehensive gene annotations to var dataframe.
    
    Parameters:
    -----------
    adata : AnnData
        Annotated data object
    ribosomal_genes_path : str
        Path to ribosomal genes file
    gene_database_path : str
        Path to gene database file
    cell_cycle_genes_path : str
        Path to cell cycle genes file
    """
    print("🧬 Adding comprehensive gene annotations...")

    # Create reference files dict for internal use
    REFERENCE_FILES = {
        "ribosomal_genes": ribosomal_genes_path,
        "gene_database": gene_database_path,
        "cell_cycle_genes": cell_cycle_genes_path
    }

    # Load ribosomal genes from tab-separated file - use "HGNC ID" column
    # ribosomal_hgnc_ids = set()
    with open(REFERENCE_FILES["ribosomal_genes"]) as f:
        ribosomal_genes_list = [x.strip() for x in f]
    ribosomal_genes = set(ribosomal_genes_list)  # For fast lookup

        # header = f.readline().strip().split('\t')
        # hgnc_col = header.index("HGNC ID")
        # for line in f:
        #     cols = line.strip().split('\t')
        #     if len(cols) > hgnc_col:
        #         ribosomal_hgnc_ids.add(cols[hgnc_col])

    print(f"  📋 Loaded {len(ribosomal_genes)} ribosomal gene")
    print(f"  📋 Sample ribosomal gene: {list(ribosomal_genes)[:5]}")

    # Load gencode database
    db = gffutils.FeatureDB(REFERENCE_FILES["gene_database"])

    # Initialize annotation columns
    adata.var['gene_type'] = ''
    # adata.var['hgnc_id'] = ''
    adata.var['gene'] = ''  # Gene symbols
    adata.var['chromosome'] = ''
    adata.var['ribosomal'] = False
    adata.var['cell_cycle'] = False
    adata.var['cell_cycle_phase'] = ''
    adata.var['pct_cells_expressed'] = 0.0

    # Calculate % cells expressed (using expression threshold > 0)
    print("  📊 Calculating % cells expressed...")
    cells_expressed = (adata.X > 0).sum(axis=0)
    if hasattr(cells_expressed, 'A1'):
        cells_expressed = cells_expressed.A1
    adata.var['pct_cells_expressed'] = (cells_expressed / adata.n_obs) * 100

    # Load cell cycle genes for annotation
    with open(REFERENCE_FILES["cell_cycle_genes"]) as f:
        cell_cycle_genes_list = [x.strip() for x in f]
    cell_cycle_genes = set(cell_cycle_genes_list)  # For fast lookup
    s_phase_genes = set(cell_cycle_genes_list[:43])  # First 43 are S phase

    # Annotate from gencode database first
    print("  🗃️  Loading gene annotations from gencode database...")
    print(f"  🔍 Sample gene IDs from adata.var.index: {list(adata.var.index[:5])}")

    annotated_count = 0
    missing_count = 0
    error_count = 0

    for i, gene_id in enumerate(adata.var.index):
        if i < 5:  # Debug first 5 genes
            print(f"  🔍 Trying to annotate gene {i+1}: {gene_id}")

        try:
            # Query database for this gene by ID
            gene = db[gene_id]
        except gffutils.exceptions.FeatureNotFoundError:
            gene = db[gene_id.split(".", 1)[0]]
            
            if i < 5:
            
                print(f"    ✅ Found gene {gene_id} in database")

            # Extract annotations - handle missing attributes gracefully
            # NOTE: We use .get() here because not all genes have all attributes (e.g., some lack hgnc_id)
            # This is expected and OK - we should NOT raise errors for missing optional attributes
            gene_type = gene.attributes.get('gene_biotype', [''])[0]
            #hgnc_id = gene.attributes.get('hgnc_id', [''])[0]
            gene_name = gene.attributes.get('gene_name', [''])[0]

            if i < 5:
                print(f"    📋 Gene info: type={gene_type}, name={gene_name}, chr={gene.seqid}")

            # Update annotations
            adata.var.loc[gene_id, 'gene_type'] = gene_type
            # adata.var.loc[gene_id, 'hgnc_id'] = hgnc_id
            adata.var.loc[gene_id, 'gene'] = gene_name  # Add gene symbols
            adata.var.loc[gene_id, 'chromosome'] = gene.seqid

            # Check ribosomal genes (by HGNC ID)
            if gene_name in ribosomal_genes:
                adata.var.loc[gene_id, 'ribosomal'] = True

            # Check cell cycle genes (by gene name)
            if gene_name in cell_cycle_genes:
                adata.var.loc[gene_id, 'cell_cycle'] = True
                # Determine phase using pre-computed S phase set
                if gene_name in s_phase_genes:
                    adata.var.loc[gene_id, 'cell_cycle_phase'] = 'S'
                else:
                    adata.var.loc[gene_id, 'cell_cycle_phase'] = 'G2M'

            annotated_count += 1

        except Exception as e:
            raise RuntimeError(f"Failed to annotate gene {gene_id}: {e}") from e

    print(f"  📊 Annotation results: {annotated_count} success, {missing_count} missing, {error_count} errors")

    # CRITICAL: Fail if gene annotation completely failed
    if annotated_count == 0:
        raise RuntimeError(f"CRITICAL: Gene annotation completely failed. 0/{adata.n_vars} genes annotated. Database lookup is broken.")

    # CRITICAL: Fail if most genes failed annotation
    success_rate = annotated_count / adata.n_vars
    if success_rate < 0.5:  # Less than 50% success
        raise RuntimeError(f"CRITICAL: Gene annotation mostly failed. Only {success_rate:.1%} ({annotated_count}/{adata.n_vars}) genes annotated successfully.")

    # Calculate mitochondrial genes using annotated gene symbols
    print("  🧬 Calculating mitochondrial gene scores...")
    mt_genes = adata.var["gene"].str.startswith(("MT-", "mt-"), na=False)
    print(f"  📊 Found {mt_genes.sum()} mitochondrial genes")

    if mt_genes.sum() > 0:
        mt_gene_names = adata.var_names[mt_genes].tolist()

        print(f"  ✅ Added MT percentage and score using {len(mt_gene_names)} genes")
    else:
        raise RuntimeError("CRITICAL: No mitochondrial genes found for scoring. MT genes are required for quality control.")

    # Calculate ribosomal scores using annotated genes
    print("  🧬 Calculating ribosomal gene scores...")
    ribo_genes = adata.var['ribosomal'] == True
    print(f"  📊 Found {ribo_genes.sum()} ribosomal genes")

    if ribo_genes.sum() > 0:
        ribo_gene_names = adata.var_names[ribo_genes].tolist()

        # Calculate ribosomal percentage
        adata.obs['pct_counts_ribo'] = (adata[:, ribo_genes].X.sum(axis=1) / adata.X.sum(axis=1)) * 100
        if hasattr(adata.obs['pct_counts_ribo'], 'A1'):
            adata.obs['pct_counts_ribo'] = adata.obs['pct_counts_ribo'].A1

        print(f"  ✅ Added ribosomal percentage and score using {len(ribo_gene_names)} genes")
    else:
        raise RuntimeError("CRITICAL: No ribosomal genes found for scoring. Ribosomal gene metrics are required for quality control.")

    # Summary statistics
    print(f"  ✅ Annotated {annotated_count}/{adata.n_vars} genes")
    print(f"  📊 Gene types: {adata.var['gene_type'].value_counts().head().to_dict()}")
    print(f"  📊 Ribosomal: {adata.var['ribosomal'].sum()}")
    print(f"  📊 Cell cycle: {adata.var['cell_cycle'].sum()}")
    print(f"  📊 Chromosomes: {adata.var['chromosome'].nunique()} unique")
    print(f"  📊 Mean % cells expressed: {adata.var['pct_cells_expressed'].mean():.1f}%")


In [10]:
import scanpy as sc
import gffutils
import numpy as np

gex_h5ad="/scratch/users/shshanhe/eRZ58_20A_analysis/eRZ58:eRZ58_GEX20A/kb_all_main_raw/counts_unfiltered_modified/adata.h5ad"
guide_h5ad="/scratch/users/shshanhe/eRZ58_20A_analysis/eRZ58:eRZ58_gRNA20A/kb_guide_main_raw/counts_unfiltered_modified/adata.h5ad"
gene_database_path = "/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/Mus_musculus.GRCm39.108.annotation.db"
db = gffutils.FeatureDB(gene_database_path)
ribosomal_genes_path="/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/genes_list_ribosomal.filtered.txt"
cell_cycle_genes_path="/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/regev_lab_cell_cycle_genes_convert_to_mouse.txt"


adata = sc.read_h5ad(gex_h5ad)
print("Loaded adata:", adata)
print("var index example:", adata.var_names[:5].tolist())


Matplotlib is building the font cache; this may take a moment.


Loaded adata: AnnData object with n_obs × n_vars = 400356 × 34183
    layers: 'ambiguous', 'mature', 'nascent'
var index example: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
🧬 Adding comprehensive gene annotations...
  📋 Loaded 182 ribosomal gene
  📋 Sample ribosomal gene: ['Mrpl41', 'Mrps28', 'Mrps10', 'Rps23', 'Rpl39l']
  📊 Calculating % cells expressed...
  🗃️  Loading gene annotations from gencode database...
  🔍 Sample gene IDs from adata.var.index: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
  🔍 Trying to annotate gene 1: ENSMUSG00000086053.2


RuntimeError: Failed to annotate gene ENSMUSG00000086053.2: ENSMUSG00000086053.2

In [14]:
add_comprehensive_gene_annotations(
    adata,
    ribosomal_genes_path,
    gene_database_path,
    cell_cycle_genes_path)

🧬 Adding comprehensive gene annotations...
  📋 Loaded 182 ribosomal gene
  📋 Sample ribosomal gene: ['Mrpl41', 'Mrps28', 'Mrps10', 'Rps23', 'Rpl39l']
  📊 Calculating % cells expressed...
  🗃️  Loading gene annotations from gencode database...
  🔍 Sample gene IDs from adata.var.index: ['ENSMUSG00000086053.2', 'ENSMUSG00000100764.2', 'ENSMUSG00000102095.2', 'ENSMUSG00000100635.2', 'ENSMUSG00000100480.2']
  🔍 Trying to annotate gene 1: ENSMUSG00000086053.2
    ✅ Found gene ENSMUSG00000086053.2 in database
    📋 Gene info: type=lncRNA, name=Gm15178, chr=1
  🔍 Trying to annotate gene 2: ENSMUSG00000100764.2
    ✅ Found gene ENSMUSG00000100764.2 in database
    📋 Gene info: type=lncRNA, name=Gm29155, chr=1
  🔍 Trying to annotate gene 3: ENSMUSG00000102095.2
    ✅ Found gene ENSMUSG00000102095.2 in database
    📋 Gene info: type=lncRNA, name=C730036E19Rik, chr=1
  🔍 Trying to annotate gene 4: ENSMUSG00000100635.2
    ✅ Found gene ENSMUSG00000100635.2 in database
    📋 Gene info: type=lncRNA, 

  📊 Annotation results: 34183 success, 0 missing, 0 errors
  🧬 Calculating mitochondrial gene scores...
  📊 Found 13 mitochondrial genes
  ✅ Added MT percentage and score using 13 genes
  🧬 Calculating ribosomal gene scores...
  📊 Found 176 ribosomal genes
  ✅ Added ribosomal percentage and score using 176 genes
  ✅ Annotated 34183/34183 genes
  📊 Gene types: {'protein_coding': 21959, 'lncRNA': 11527, 'IG_V_gene': 218, 'IG_V_pseudogene': 158, 'TR_V_gene': 144}
  📊 Ribosomal: 176
  📊 Cell cycle: 97
  📊 Chromosomes: 35 unique
  📊 Mean % cells expressed: 0.0%


In [3]:
# Compare overlap between two ribosomal gene lists (one gene symbol per line)

from pathlib import Path

f1 = Path("/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/human_ribosomal_genes_convert_to_mouse.txt")
f2 = Path("/oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/genes_list_ribosomal.filtered.txt")

# read + clean (drop empty lines, strip whitespace, drop comments)
s1 = {line.strip() for line in f1.read_text().splitlines() if line.strip() and not line.strip().startswith("#")}
s2 = {line.strip() for line in f2.read_text().splitlines() if line.strip() and not line.strip().startswith("#")}

overlap = s1 & s2
only1 = s1 - s2
only2 = s2 - s1

print("File 1:", f1)
print("  n =", len(s1))
print("File 2:", f2)
print("  n =", len(s2))
print("\nOverlap:")
print("  n =", len(overlap))

# show a few examples
print("\nExamples of overlap (up to 30):")
print(sorted(list(overlap))[:30])

print("\nOnly in file 1 (up to 30):")
print(sorted(list(only1))[:30])

print("\nOnly in file 2 (up to 30):")
print(sorted(list(only2))[:30])

File 1: /oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/human_ribosomal_genes_convert_to_mouse.txt
  n = 181
File 2: /oak/stanford/groups/engreitz/Users/shshanhe/project/exchange_perturb_project/data/reference/genes_list_ribosomal.filtered.txt
  n = 182

Overlap:
  n = 163

Examples of overlap (up to 30):
['Fau', 'Mrpl1', 'Mrpl10', 'Mrpl11', 'Mrpl12', 'Mrpl13', 'Mrpl14', 'Mrpl15', 'Mrpl16', 'Mrpl17', 'Mrpl18', 'Mrpl19', 'Mrpl2', 'Mrpl20', 'Mrpl21', 'Mrpl22', 'Mrpl23', 'Mrpl24', 'Mrpl27', 'Mrpl28', 'Mrpl3', 'Mrpl30', 'Mrpl32', 'Mrpl33', 'Mrpl34', 'Mrpl35', 'Mrpl36', 'Mrpl37', 'Mrpl38', 'Mrpl39']

Only in file 1 (up to 30):
['Aurkaip1', 'Chchd1', 'Dap3', 'Gadd45gip1', 'Gm6133', 'Gm6525', 'Ptcd3', 'Rpl27rt', 'Rpl34-ps1', 'Rpl36-ps4', 'Rpl36a-ps1', 'Rpl37rt', 'Rpl9-ps6', 'Rplp1rt', 'Rplp2-ps1', 'Rps18-ps5', 'Rps18-ps6', 'Rps27rt']

Only in file 2 (up to 30):
['Mrps36', 'Rpl32l', 'Rpl6l', 'Rps12l1', 'Rps19bp1', 'Rps23rg1', 'Rps3a2', 'Rps3a3', 'Rp